# Intro

This notebook demonstrates basic usage of the firefly_client API through Python so users can visualize data output without having Firefly installed.

Note that it may be necessary to wait for some cells (like those displaying an image) to complete before executing later cells.

## Repository Setup

Download the folder for firefly_client using git while in bash shell (on Mac or Linux) or Anaconda Prompt app (Windows):

git clone https://github.com/Caltech-IPAC/firefly_client

## Notebook Setup

Call the imports for firefly_client:

In [ ]:
from firefly_client import FireflyClient

The firefly_client API involves a server in place of having a local install of Firefly on the computer being used. In this example, we use the IRSA Viewer application for the server:

In [ ]:
url = 'https://irsa.ipac.caltech.edu/irsaviewer'

Instantiate `FireflyClient` using the URL above, which in this case involves parameters for WebSocket channel ID ("wschannel") and Slate HTML acting as landing page (can choose to have none for following case):

In [ ]:
fc = FireflyClient(url, "wschannel", html_file='slate.html')

#fc = FireflyClient(url, "wschannel", html_file=None)

## Download Sample Data

For the following example, start the process to download a sample image and table by gathering astropy imports for data utility (as firefly_client can use astropy for convenience but is not itself dependent on it):

In [ ]:
import astropy.utils.data

Download a cutout of a WISE band as 1 FITS image:

In [ ]:
image_url = ('http://irsa.ipac.caltech.edu/ibe/data/wise/allsky/4band_p1bm_frm/6a/02206a' + 
             '/149/02206a149-w1-int-1b.fits?center=70,20&size=200pix')
filename = astropy.utils.data.download_file(image_url, cache=True, timeout=120)

Download a 2MASS catalog using an IRSA VO Table Access Protocol ([TAP](https://irsa.ipac.caltech.edu/docs/program_interface/astropy_TAP.html)) search:

In [ ]:
table_url = ("http://irsa.ipac.caltech.edu/TAP/sync?FORMAT=IPAC_TABLE&" +
                                             "QUERY=SELECT+*+FROM+fp_psc+WHERE+CONTAINS(POINT('J2000',ra,dec)," +
                                             "CIRCLE('J2000',70.0,20.0,0.1))=1")
tablename = astropy.utils.data.download_file(table_url, timeout=120, cache=True)

## Display Tables and Catalogs

Open a browser to the firefly server in a new tab. Note that the browser open only works when running the notebook locally, otherwise a link is displayed. Test out whether starting a browser tab with IRSA Viewer as the firefly server is possible below:

In [ ]:
localbrowser, browser_url = fc.launch_browser()

If it does not open automatically, try using the following command to display a web browser link to click on:

In [ ]:
fc.display_url()

Alternatively, uncomment the lines below (while commenting out the 2 lines above) to display the browser application in the notebook:

In [ ]:
#from IPython.display import IFrame
#print('url: %s' % fc.get_firefly_url())
#IFrame(fc.get_firefly_url(), 1100, 1000)

The current example involves 2 images in FITS format to be viewed through the chosen browser, both in different ways.
For the 1st method, upload a local file to the server and then display it (while keeping in mind the `plot_id` parameter chosen):

In [ ]:
imval = fc.upload_file(filename)
status = fc.show_fits(file_on_server=imval, plot_id="wise-cutout", title='WISE Cutout')

For the 2nd method, pull an image from a URL for `show_fits` to display with a different parameter than above:

In [ ]:
status = fc.show_fits(file_on_server=None, plot_id="wise-fullimage", 
                      URL='http://irsa.ipac.caltech.edu/ibe/data/wise/allsky/4band_p1bm_frm/6a/02206a' + 
             '/149/02206a149-w1-int-1b.fits')

Upload a table from a catalog (with coordinates) to the server so it will overlay the image by default. Note that in the case being looked at, markers can be visible on both "wise-cutout" and "wise-fullimage" (after checking on one's own time) since they would include the same field of view:

In [ ]:
file= fc.upload_file(tablename)
status = fc.show_table(file, tbl_id='tablemass', title='My 2MASS Catalog', page_size=50)

Try to add an xy plot using the uploaded table data: ((WIP Note: Error to fix!))

In [ ]:
status = fc.show_xyplot(tbl_id='tablemass', xCol='j_m', yCol='h_m-k_m')

Zoom into the full image by a factor of 2 while noting the `plot_id` parameter needed:

In [ ]:
status = fc.set_zoom('wise-fullimage', 2)

Pan the full image to center on a given celestial coordinate: ((WIP Note: Error to fix!))

In [ ]:
status = fc.set_pan('wise-fullimage', x=70, y=20, coord='J2000')

Set the stretch for the full image based on IRAF zscale interval with a linear algorithm:

In [ ]:
status = fc.set_stretch('wise-fullimage', stype='minmax', algorithm='linear')

Add region data to the cutout image (2 areas to begin with):

In [ ]:
my_regions= ['image;polygon 125 25 160 195 150 150 #color=cyan',
            'icrs;circle 69.95d 20d 30i # color=orange text={region 5/7}']
status = fc.add_region_data(region_data=my_regions, region_layer_id='layer1',
                  plot_id='wise-cutout')

Remove the second region whlie keeping the first one visible:

In [ ]:
fc.remove_region_data(region_data=my_regions[1], region_layer_id='layer1')